<a href="https://colab.research.google.com/github/singularity014/Transformer_NeuroMachineTranslation/blob/master/nmt_transformer_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) import dependencies


In [11]:
import time
import numpy as np
import math
import re

from google.colab import drive

In [12]:
import tensorflow as tf
tf.__version__

'2.3.0'

In [13]:
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [14]:
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# II) Load Data 


In [40]:
# English (Translated)
with open("/content/drive/My Drive/nmt_data/europarl-v7.fr-en.en", 'r', encoding='utf-8') as f:
    europarl_en = f.read()

# French (Original)
with open("/content/drive/My Drive/nmt_data/europarl-v7.fr-en.fr", 'r', encoding='utf-8') as f:
    europarl_fr = f.read()

# Prefixes English     
with open("/content/drive/My Drive/nmt_data/nonbreaking_prefix.en", 'r', encoding='utf-8') as f:
    non_breaking_prefix_en = f.read()

# Prefixes French 
with open("/content/drive/My Drive/nmt_data/nonbreaking_prefix.fr",
          mode='r',
          encoding='utf-8') as f:
    non_breaking_prefix_fr = f.read()

## Cleaning data

In [41]:
non_breaking_prefix_en = non_breaking_prefix_en.split("\n")
non_breaking_prefix_en = [' ' + pref + '.' for pref in non_breaking_prefix_en]
non_breaking_prefix_fr = non_breaking_prefix_fr.split("\n")
non_breaking_prefix_fr = [' ' + pref + '.' for pref in non_breaking_prefix_fr]